# Differential  analysis between surgery/non-surgery group using Ancom
Since Skbio is only available in python3, this is a separate notebook

In [1]:
from skbio.stats.composition import ancom
import pandas as pd 
import scipy

## Test differential abundance of species between two groups: surgery, non-sergery

In [2]:
#first load species abundance and metadata 
df_meta=pd.read_csv('./data/metadata_updated_filtered_201909.csv',index_col=0)
dic_surgery=dict(zip(df_meta.index,df_meta.surgery_type))
df_species=pd.read_csv('./data/metaphlan2_species_level_abundance_classified.csv',index_col=0,header=0)

#replace 0 entry in species abundance with a small number 
df_species=df_species.replace(to_replace=0,value=0.00000000001)
df_species=df_species.T


In [3]:
#seprate samples into surgery and non-surgery groups
label_2group=[]
for i in df_species.index: 
    if dic_surgery[i]=='none': 
        label_2group.append(0)
    else: 
        label_2group.append(1)
label_2group

#run ancom test 
grouping = pd.Series(label_2group,index=df_species.index)
results_2group = ancom(df_species, grouping,multiple_comparisons_correction='holm-bonferroni',significance_test=scipy.stats.mannwhitneyu)

#filter for the top10 significant features
df_2group=results_2group[0][results_2group[0]['Reject null hypothesis']==True].sort_values(by='W',ascending=False)


### Perform center log-ratio transformations on the top 10 species 

In [4]:
from skbio.stats.composition import clr

#extract the species abundance of the top10 species 
df_top10=df_species[df_2group.index[0:10]]
df_top10_clr=pd.DataFrame(data=clr(df_top10),index=df_top10.index,columns=df_top10.columns)
#df_top10_clr.to_csv('./data/top10_differentiating_species_clr.csv')

## Test differential abundance of functions between two groups: surgery, non-sergery

In [5]:
#first load pathway abundance and metadata 
df_pathway=pd.read_csv('./data/Filtered_normalized_cpm_pathways_new_unstratified.csv',index_col=0)
print (df_pathway.shape)
df_pathway=df_pathway.T
df_pathway=df_pathway.fillna(value=0)

#replace 0 entry in pathway abundance with a small number 
df_pathway=df_pathway.replace(to_replace=0,value=0.00000000001)

(483, 300)


In [6]:
#us the one way f-test 
label_2group_pw=[]
for i in df_pathway.index: 
    if dic_surgery[i]=='none': 
        label_2group_pw.append(0)
    else: 
        label_2group_pw.append(1)
label_2group_pw

#run ancom
grouping_pw = pd.Series(label_2group_pw,index=df_pathway.index)
results_2group_pw = ancom(df_pathway, grouping_pw,multiple_comparisons_correction='holm-bonferroni')

#filter for the top10 significant features
df_2group_pw=results_2group_pw[0][results_2group_pw[0]['Reject null hypothesis']==True].sort_values(by='W',ascending=False)


/usr/local/lib/python3.4/dist-packages/scipy-1.1.0rc1-py3.4-linux-x86_64.egg/scipy/stats/stats.py:2933: RuntimeWarning: invalid value encountered in double_scalars
  f = msb / msw


### Perform center log-ratio transformations on the top 10 species 

In [7]:
from skbio.stats.composition import clr

#extract the species abundance of the top10 pathways 
df_top10_pw=df_pathway[df_2group_pw.index[0:10]]
df_top10_clr_pw=pd.DataFrame(data=clr(df_top10_pw),index=df_top10_pw.index,columns=df_top10_pw.columns)
#df_top10_clr_pw.to_csv('./data/top10_differentiating_pathway_clr.csv')